## Alcholic Anonymous News articles extraction from Hindustan Times. Data Extraction of following parameters
- Headline
- Description
- Author
- Published Date
- Category
- Publication
- News
- URL
- Keywords
- Summary

### Importing the necessary Libraries

In [1]:
from selenium.webdriver.chrome.options import Options # enables options in web browser
from selenium import webdriver # web-based automation tool for Python
from newspaper import Article # Article scraping & curation
from bs4 import BeautifulSoup # Python library for pulling data out of HTML and XML files
from requests import get # standard for making HTTP requests in Python
import pandas as pd # library written for data manipulation and analysis
import sys, time #  System-specific parameters and functions

### Creating Empty lists for Alcholic Anonymous News Articles parameters data to be extracted

In [2]:
headlines, descriptions, dates, authors, news, keywords, summaries, urls, category, publication = [], [], [], [], [], [], [], [], [], []

### Finding the total no.of.pages by total no.of articles from google search results¶

In [3]:
keyword = 'Alcoholics Anonymous site:www.hindustantimes.com'

url = 'https://www.google.com/search?q=' + '+'.join(keyword.split())

soup = BeautifulSoup(get(url).text, 'lxml')
try:
    # Extracts the digits if it the resulted number without comma ','. eg: About 680 results (0.23 seconds)
    max_pages = round([int(s) for s in soup.select_one('div#resultStats').text.split() if s.isdigit()][0]/10)
    max_pages = max_pages + 1
except:
    # Extracts the digits if it the resulted number without comma ','. eg: About 1,080 results (0.23 seconds)
    max_pages = round(int(''.join(i for i in soup.select_one('div#resultStats').text if i.isdigit()))/10)
    max_pages = max_pages + 1

### Iterates max_pages value through while loop. Scraping the Articles urls

In [4]:
options = Options()
options.headless = True
browser = webdriver.Chrome(options=options)
browser.get(url)

index = 0

while True:
    try:
        index +=1
        page = browser.page_source
        soup = BeautifulSoup(page, 'lxml')
        linky = [soup.select('.r')[i].a['href'] for i in range(len(soup.select('.r')))]
        urls.extend(linky)
        if index == max_pages:
            break
        browser.find_element_by_xpath('//*[@id="pnnext"]/span[2]').click()
        time.sleep(2)
        sys.stdout.write('\r' + str(index) + ' : ' + str(max_pages) + '\r')
        sys.stdout.flush()
    except:
        pass
    
browser.quit()

### To remove duplicates urls entries in the list by executing below line

In [5]:
urls = list(dict.fromkeys(urls))
print("Total Extracted URL's are" + ' : ' + str(len(urls)), type(urls))

Total Extracted URL's are : 60 <class 'list'>


### Iterates urls through for loop. Scraping the Articles with above parameters

In [6]:
%%time
for index, url in enumerate(urls):
    try:
        # Parse the url to NewsPlease 
        soup = BeautifulSoup(get(url).text, 'lxml')
        article = Article(url)
        article.download()
        article.parse()
        article.nlp()
        
        # Extracts the Headlines
        try:
            try:
                headlines.append(article.title.strip())
            except:
                headlines.append(soup.select_one('meta[property="og:title"]')['content'].strip())
        except:
            headlines.append(None)
            
        # Extracts the Descriptions    
        try:
            try:
                descriptions.append(soup.select_one('meta[property="og:description"]')['content'].strip().replace('\n', ' '))
            except:
                descriptions.append(article.meta_description.strip())
        except:
            descriptions.append(None)
            
        # Extracts the Authors
        try:
            try:
                authors.append(soup.select_one('meta[name="author"]')['content'].strip())
            except:
                authors.append(article.authors.strip())
        except:
            authors.append(None)
        
        # Extracts the published dates
        try:
            try:
                dates.append(soup.select_one('meta[property="article:published_time"]')['content'].strip())
            except:
                dates.append(str(article.publish_date))
        except:
            dates.append(None)
            
        # Extracts the news articles
        try:
            try:
                news.append(' '.join(soup.select_one('.storyText').text.split()).replace('\n', '').strip())
            except:
                news.append(' '.join(article.text.split()).replace("\'\'"," ").replace("\'", "").replace(" / ", ""))
        except:
            news.append(None)
            
        # Extracts the news category
        try:
            category.append(article.meta_data['category'])
        except:
            category.append(None)
            
        # Extracts the news publication
        try:
            publication.append(article.meta_data['og']['site_name'])
        except:
            publication.append(None)

        # Extracts Keywords and Summaries
        try:
            keywords.append(article.keywords)
            summaries.append(article.summary)
        except:
            keywords.append(None)
            summaries.append(None)
                        
    except:
        headlines.append(None)
        descriptions.append(None)
        authors.append(None)
        dates.append(None)
        category.append(None)
        publication.append(None)
        news.append(None)
        keywords.append(None)
        summaries.append(None)

    sys.stdout.write('\r' + str(index) + ' : ' + str(url) + '\r')
    sys.stdout.flush()

Wall time: 3min 49sndustantimes.com/dehradun/traders-in-uttarakhand-feel-heat-as-liquor-sales-miss-target/story-MH36xFPYFnWGP26hP1lQ3M.htmlmawHtjPcsKBL.html.htmlyyHHiV4M.html


### Checking Array Length of each list to create DataFrame

In [7]:
print(len(headlines), len(descriptions), len(authors), len(dates), len(category), len(publication), len(news), len(keywords), len(summaries), len(urls))

60 60 60 60 60 60 60 60 60 60


### Creating a csv file after checking array length and droping the missing values from the dataset

In [8]:
if len(headlines) == len(descriptions) == len(authors) == len(dates) == len(news) == len(publication) == len(keywords) == len(summaries) == len(urls) == len(category):
    tbl = pd.DataFrame({'Headlines' : headlines,
                        'Descriptions' : descriptions,
                        'Authors' : authors,
                        'Published_Dates' : dates,
                        'Publication' : publication,
                        'Articles' : news,
                        'category' : category,
                        'Keywords' : keywords,
                        'Summaries' : summaries,
                        'Source_URLs' : urls})
    tbl.dropna()
    path = 'C:\\Users\\GM\\Desktop\\!Code!\\CDRI\\Alcoholics Anonymous\\Data Extraction\\#Dataset\\'
    tbl.to_csv(path+'Hindustan_Times.csv', index=False)
else:
    print('Array lenght does not match!')

tbl.head()

,Headlines,Descriptions,Authors,Published_Dates,Publication,Articles,category,Keywords,Summaries,Source_URLs
0,"Women alcoholics on the rise in Delhi, and acr...","States like Kerala, Andhra Pradesh, Manipur, M...",None,2017-12-10T08:05Z,https://www.hindustantimes.com/,When a sexagenarian wobbled up to the podium a...,{},"[alcoholics, think, rise, help, drink, problem...","According to members of Shakti, the Delhi grou...",https://www.hindustantimes.com/delhi-news/wome...
1,"Narcotics Anonmyous helps kick drug addiction,...","Narcotics Anonymous is a non-profit, internati...",None,2015-12-06T14:10Z,https://www.hindustantimes.com/,He was only eight when his father handed him h...,{},"[meetings, addiction, members, hands, helps, l...",That was till he found solace in Narcotics Ano...,https://www.hindustantimes.com/delhi/narcotics...
2,The destroyer of all that is… good,"Fire gives us warmth, but it also burns us. We...",None,2012-07-01T23:11Z,https://www.hindustantimes.com/,"Fire gives us warmth, but it also burns us. We...",{},"[unable, consumed, start, cut, drink, person, ...",Eye-Opener: Have you now reached a stage in yo...,https://www.hindustantimes.com/columns/the-des...
3,Benefits of AA Meetings,The fact that many have won the battle over th...,None,2003-06-28T21:20Z,https://www.hindustantimes.com/,You are among people who have or had problems ...,{},"[meetings, inter, aa, helps, anonymous, aloud,...",You are among people who have or had problems ...,https://www.hindustantimes.com/health-and-fitn...
4,Drink habits: A toast to the new cocktails,In a large-sample study conducted in Karnataka...,None,2008-05-11T00:42Z,https://www.hindustantimes.com/,"A wife who drinks spirituous liquor, is of bad...",{},"[thats, cocktails, wife, power, toast, aa, dri...","She says haltingly, “Though I had been drinkin...",https://www.hindustantimes.com/entertainment/d...


In [9]:
tbl.shape

(60, 10)